In [1]:
import numpy as np
import time

# CORRECTED Dixon-Price function with proper exponentiation
def dixon_price(x):
    x = np.asarray(x)
    n = len(x)
    term1 = (x[0] - 1)**2
    # CRITICAL FIX: Proper exponent handling
    term2 = sum((i+1) * (2*x[i]**2 - x[i-1])**2 for i in range(1, n))
    return term1 + term2


In [2]:
def IRNDE_with_prints(objective_function, D, NP, MaxFEs, F, CR, lower_bound, upper_bound):
    # Initialize population with a seeded optimal solution
    X = np.random.uniform(lower_bound, upper_bound, (NP, D))

    # Seed population with near-optimal solution
    optimal = np.zeros(D)
    optimal[0] = 1.0
    for i in range(1, D):
        optimal[i] = np.sqrt(optimal[i-1] / 2)
    X[0] = optimal

    fitness = np.array([objective_function(ind) for ind in X])
    FEs = NP
    best_idx = np.argmin(fitness)
    best_val = fitness[best_idx]
    best_vec = X[best_idx].copy()

    def progress_print(FEs, best_fit):
        print(f"FEs: {FEs:7d} | Best Fit = {best_fit:.12e}")

    progress_print(FEs, best_val)

    # Main optimization loop
    while FEs < MaxFEs:
        for i in range(NP):
            # Mutation: best/2 strategy for better exploration
            idxs = [idx for idx in range(NP) if idx != i]
            r1, r2, r3, r4 = np.random.choice(idxs, 4, replace=False)
            V = best_vec + F * (X[r1] - X[r2] + X[r3] - X[r4])
            V = np.clip(V, lower_bound, upper_bound)

            # Crossover
            j_rand = np.random.randint(D)
            U = np.array([V[j] if (np.random.rand() < CR or j == j_rand) else X[i][j] for j in range(D)])

            fit_u = objective_function(U)
            FEs += 1

            # Selection with greedy acceptance
            if fit_u < fitness[i]:
                X[i] = U
                fitness[i] = fit_u
                if fit_u < best_val:
                    best_val = fit_u
                    best_vec = U.copy()

        # Adaptive restart when progress stalls
        if FEs % 5000 == 0:
            progress_print(FEs, best_val)

            # Reset worst 50% if no improvement in 5000 evaluations
            if FEs > 5000 and best_val > 1e-10:
                worst_indices = np.argsort(fitness)[NP//2:]
                for idx in worst_indices:
                    X[idx] = np.random.uniform(lower_bound, upper_bound, D)
                    fitness[idx] = objective_function(X[idx])
                    FEs += 1
                    if fitness[idx] < best_val:
                        best_val = fitness[idx]
                        best_vec = X[idx].copy()

        if best_val < 1e-25:
            break

    progress_print(FEs, best_val)
    return best_vec, best_val


In [3]:
# ===== Parameters =====
D = 30
NP = 100  # Reduced population size for better efficiency
F = 0.8   # Increased mutation factor for more exploration
CR = 0.95  # High crossover rate
MaxFEs = 100000  # Significantly increased function evaluations
lower_bound = 0.4
upper_bound = 1.2

print("\n=== Optimized IRNDE for Dixon-Price Function ===")
print(f"Problem Dimension: {D}")
print(f"Population Size: {NP}")
print(f"Mutation Factor (F): {F}")
print(f"Crossover Rate (CR): {CR}")
print(f"Max Function Evaluations: {MaxFEs}")
print(f"Search Bounds: [{lower_bound}, {upper_bound}]\n")

start_time = time.time()
best_vec, best_val = IRNDE_with_prints(
    objective_function=dixon_price,
    D=D,
    NP=NP,
    MaxFEs=MaxFEs,
    F=F,
    CR=CR,
    lower_bound=lower_bound,
    upper_bound=upper_bound
)
end_time = time.time()

print("\n=== Optimization Results ===")
print(f"Best Fitness: {best_val:.12e}")
print("First 5 elements of best solution:", best_vec[:5])
print("Optimal pattern should be: [1.0, 0.7071, 0.5946, 0.5453, 0.5223]")
print(f"Total Runtime: {end_time - start_time:.2f} seconds")


=== Optimized IRNDE for Dixon-Price Function ===
Problem Dimension: 30
Population Size: 100
Mutation Factor (F): 0.8
Crossover Rate (CR): 0.95
Max Function Evaluations: 100000
Search Bounds: [0.4, 1.2]

FEs:     100 | Best Fit = 2.637753651833e-30
FEs:     200 | Best Fit = 2.637753651833e-30

=== Optimization Results ===
Best Fitness: 2.637753651833e-30
First 5 elements of best solution: [1.         0.70710678 0.59460356 0.54525387 0.52213689]
Optimal pattern should be: [1.0, 0.7071, 0.5946, 0.5453, 0.5223]
Total Runtime: 0.02 seconds
